![Hugging Face](https://miro.medium.com/max/2000/1*Z4mGaMsu34LfyE76QAi9qA.png)


# Using HuggingFace

In this tutorial, we will learn how to use the various functionalities offered by [HuggingFace](https://huggingface.co/).

## About Hugging Face

- HuggingFace is 'On a mission to solve NLP, One commit at a time', as per their tagline.

- The HuggingFace Transformer library provides state-of-the-art Transformer Based Models, their pretrained weights and a lots more

In [ ]:
#!pip install transformers

# Pipeline

https://github.com/huggingface/transformers#quick-tour-of-pipelines

In [ ]:
from transformers import pipeline

## Sentiment Analysis

In [ ]:
nlp = pipeline('sentiment-analysis')
nlp('We are very happy to include pipeline into the transformers repository.')

## Question Answering

In [ ]:
nlp = pipeline('question-answering')
nlp({
    'question': 'What is my name ?',
    'context': 'My name is Ahmed, I am working with HuggingFace'
})

## Predicting Masks

In [ ]:
nlp = pipeline('fill-mask')
nlp('I hope you <mask> this video')

## NER

In [ ]:
nlp = pipeline('ner')
nlp('It is me, Ahmed, I am working with HuggingFace')

# Text Generation

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
tokeniser = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

## Single Word Prediction

In [ ]:
device = "cuda"

text = "let us see how this turns"
indexed_tokens = tokeniser.encode(text)
tokens_tensor = torch.tensor([indexed_tokens])
model.eval()
tokens_tensor = tokens_tensor.to(device)
model.to(device)

In [ ]:
with torch.no_grad():
  outputs = model(tokens_tensor)
  predictions = outputs[0]

print(outputs[0].shape)

predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokeniser.decode(indexed_tokens + [predicted_index])

print(predicted_text)

## Looping for multi-word

In [ ]:
chars = 0
text = "i am very exited to present to you this"
while chars<50:
  chars += 1
  indexed_tokens = tokeniser.encode(text)
  tokens_tensors = torch.tensor([indexed_tokens])
  tokens_tensors = tokens_tensors.to(device)
  with torch.no_grad():
    outputs = model(tokens_tensors)
    predictions = outputs[0]
  predicted_index = torch.argmax(predictions[0,-1,:]).item()
  text = tokeniser.decode(indexed_tokens + [predicted_index])

print(text)

# Summarising text using HuggingFace

In [ ]:
text = 'Shakespeare occupies a position unique in world literature. Other poets, such as Homer and Dante, and novelists, such as Leo Tolstoy and Charles Dickens, have transcended national barriers, but no writer’s living reputation can compare to that of Shakespeare, whose plays, written in the late 16th and early 17th centuries for a small repertory theatre, are now performed and read more often and in more countries than ever before. The prophecy of his great contemporary, the poet and dramatist Ben Jonson, that Shakespeare “was not of an age, but for all time,” has been fulfilled. It may be audacious even to attempt a definition of his greatness, but it is not so difficult to describe the gifts that enabled him to create imaginative visions of pathos and mirth that, whether read or witnessed in the theatre, fill the mind and linger there. He is a writer of great intellectual rapidity, perceptiveness, and poetic power. Other writers have had these qualities, but with Shakespeare the keenness of mind was applied not to abstruse or remote subjects but to human beings and their complete range of emotions and conflicts. Other writers have applied their keenness of mind in this way, but Shakespeare is astonishingly clever with words and images, so that his mental energy, when applied to intelligible human situations, finds full and memorable expression, convincing and imaginatively stimulating. As if this were not enough, the art form into which his creative energies went was not remote and bookish but involved the vivid stage impersonation of human beings, commanding sympathy and inviting vicarious participation. Thus, Shakespeare’s merits can survive translation into other languages and into cultures remote from that of Elizabethan England.'

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

In [ ]:
inputs = tokenizer.batch_encode_plus([text], max_length=1024, return_tensors='pt')

summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=100, early_stopping=False)

for ids in summary_ids:
    short = tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

    print(len(text), len(short))
    print(short)